1. Write a cryptarithmetic solver. It should take as input an equation (Eg. TWO+TWO=FOUR) and return the variable assignment for the letters.

In [ ]:
import itertools

def solve_cryptarithmetic(equation):
    # Split equation into left and right parts
    left_part, right_part = equation.replace(" ", "").split("=")

    # Extract the words from the equation
    left_words = left_part.split("+")
    right_word = right_part

    # Find unique characters
    unique_chars = set("".join(left_words) + right_word)

    if len(unique_chars) > 10:
        return "Too many unique characters for digits 0-9"

    # Create a list of digits from 0 to 9
    digits = list(range(10))

    # Generate all possible digit assignments for the unique characters
    for perm in itertools.permutations(digits, len(unique_chars)):
        char_to_digit = dict(zip(unique_chars, perm))

        # Check if the assignment satisfies the equation
        left_values = [int("".join(str(char_to_digit[c]) for c in word)) for word in left_words]
        right_value = int("".join(str(char_to_digit[c]) for c in right_word))

        # Ensure no number starts with a zero
        if any(str(char_to_digit[word[0]]) == '0' for word in left_words + [right_word]):
            continue

        if sum(left_values) == right_value:
            return char_to_digit

    return "No solution found"

# Example usage
equation = "BABY + BEAR = TEDDY"
solution = solve_cryptarithmetic(equation)
print(solution)


{'D': 2, 'T': 1, 'B': 8, 'Y': 3, 'E': 7, 'A': 4, 'R': 0}


Write a Produku solver. A Produku is a type of Sudoku, except that you will have, in addition to the regular Sudoku constraints, a set of number groups, whose product, will be given more details will be given later today.

In [ ]:

import numpy as np
from itertools import product

class ProdokuSolver:
    def __init__(self, grid, products):
        self.grid = np.array(grid)
        self.products = products  # {group: product_value}

    def is_valid(self, row, col, num):
        # Check the row
        if num in self.grid[row]:
            return False
        # Check the column
        if num in self.grid[:, col]:
            return False
        # Check the 3x3 box
        box_row, box_col = row // 3 * 3, col // 3 * 3
        if num in self.grid[box_row:box_row + 3, box_col:box_col + 3]:
            return False
        return True

    def check_products(self):
        for group, expected_product in self.products.items():
            product_value = 1
            for (r, c) in group:
                if self.grid[r, c] != 0:  # Only consider filled cells
                    product_value *= self.grid[r, c]
            # Check if the product so far matches
            if product_value > expected_product:
                return False
            if all(self.grid[r, c] != 0 for (r, c) in group) and product_value != expected_product:
                return False
        return True

    def solve(self):
        empty = np.argwhere(self.grid == 0)
        if not empty.size:
            return True  # Puzzle solved

        row, col = empty[0]

        for num in range(1, 10):
            if self.is_valid(row, col, num):
                self.grid[row, col] = num
                if self.check_products():
                    if self.solve():
                        return True
                self.grid[row, col] = 0  # Backtrack

        return False

    def print_grid(self):
        for row in self.grid:
            print(" ".join(str(num) for num in row))

# Example usage
initial_grid = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

# Example product groups
products = {
    # Define groups of cells as tuples of (row, col) and their required product
    # For example, the product of the first two cells
    frozenset([(0, 0), (0, 1)]): 15  # This means grid[0][0] * grid[0][1] = 15
}

solver = ProdokuSolver(initial_grid, products)
if solver.solve():
    solver.print_grid()
else:
    print("No solution found.")



5 3 4 6 7 8 9 1 2
6 7 2 1 9 5 3 4 8
1 9 8 3 4 2 5 6 7
8 5 9 7 6 1 4 2 3
4 2 6 8 5 3 7 9 1
7 1 3 9 2 4 8 5 6
9 6 1 5 3 7 2 8 4
2 8 7 4 1 9 6 3 5
3 4 5 2 8 6 1 7 9
